In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# mypy: disable-error-code="import-not-found"

# The notebook should be executed from the project root directory
import os
import sys
from pathlib import Path

if "_correct_path" not in locals():
    os.chdir("..")
    sys.path.append(".")
    print(f"changed dir to {Path('.').resolve()})")
    _correct_path = True

In [ ]:
import os

import pandas as pd

from utils.schema import AnalystDataset

In [ ]:
dataset_url = "https://s3.amazonaws.com/datarobot_public_datasets/10k_diabetes_20.csv"

df = pd.read_csv(dataset_url)

# Create dataset dictionary
dataset = AnalystDataset(
    name=os.path.splitext(os.path.basename(dataset_url))[0], data=df.to_dict("records")
)

In [ ]:
from utils.schema import RunAnalysisResult

with open("tests/models/run_analysis_result.json") as f:
    analysis_result = RunAnalysisResult.model_validate_json(f.read())

In [ ]:
from utils.api import cleanse_dataframes

cleansed_data = await cleanse_dataframes([dataset])

In [ ]:
from utils.api import suggest_questions

suggested_questions = await suggest_questions(cleansed_data)

In [ ]:
from utils.api import get_dictionary

dictionary = await get_dictionary([dataset])

In [ ]:
from utils.api import rephrase_message
from utils.schema import ChatRequest

question = "What is the relationship between length of stay and readmission?"
chat_response = await rephrase_message(
    messages=ChatRequest(
        messages=[
            {
                "role": "user",
                "content": question,
            },
            {
                "role": "user",
                "content": question + "Please order the chart by readmission rate",
            },
        ],
    )
)

In [ ]:
chat_response

In [ ]:
from utils.api import run_analysis
from utils.schema import RunAnalysisRequest

analysis_request = RunAnalysisRequest(
    data=cleansed_data,
    dictionary=dictionary,
    question=chat_response,
)
analysis_result = await run_analysis(analysis_request)

In [ ]:
import asyncio

from utils.api import get_business_analysis, run_charts
from utils.schema import (
    DataDictionary,
    RunBusinessAnalysisRequest,
    RunChartsRequest,
)

# Prepare requests
chart_request = RunChartsRequest(
    data=analysis_result.data,
    question=chat_response,
)

business_request = RunBusinessAnalysisRequest(
    data=analysis_result.data,
    dictionary=DataDictionary.from_df(analysis_result.data.to_df()),
    question=chat_response,
)

# Create and start tasks immediately
charts_task = asyncio.create_task(run_charts(chart_request))
business_task = asyncio.create_task(get_business_analysis(business_request))

In [ ]:
import plotly.offline as pyo

from utils.schema import RunBusinessAnalysisResult, RunChartsResult

pyo.init_notebook_mode()

tasks = [charts_task, business_task]

# Wait for each task to complete
for coro in asyncio.as_completed(tasks):
    result = await coro

    # Determine which task completed by checking the result structure
    if isinstance(result, RunChartsResult) and (result.fig1 or result.fig2):
        if result.fig1:
            pyo.iplot(result.fig1)
        if result.fig2:
            pyo.iplot(result.fig2)

    elif isinstance(result, RunBusinessAnalysisResult):
        print(f"Bottom Line:\n{(result.bottom_line or '')}")

        print(f"Additional Insights:\n{result.additional_insights}")

        print("Follow-up Questions:")
        for q in result.follow_up_questions:
            print(f"- {q}")

In [ ]:
with open("tests/models/run_analysis_result.json", "w") as f:
    f.write(analysis_result.model_dump_json(indent=4))
with open("tests/models/run_charts_result.json", "w") as f:
    f.write(charts_task.result().model_dump_json(indent=4))
with open("tests/models/run_business_result.json", "w") as f:
    f.write(business_task.result().model_dump_json(indent=4))

In [ ]:
from utils.api import cleanse_dataframes, get_dictionary
from utils.database_helpers import Database

db_tables = Database.get_tables()
db_datasets = Database.get_data(*db_tables)
db_dataset_cleansed = await cleanse_dataframes(db_datasets)
db_dictionaries = await get_dictionary(db_datasets)

In [ ]:
from utils.api import suggest_questions

suggested_questions = await suggest_questions(db_dataset_cleansed)

In [ ]:
suggested_questions

In [ ]:
from utils.api import run_database_analysis
from utils.schema import RunDatabaseAnalysisRequest

db_run_analysis = await run_database_analysis(
    RunDatabaseAnalysisRequest(
        data=db_dataset_cleansed,
        dictionary=db_dictionaries,
        question="How does loan default rate relate to type of loan?",
    )
)

In [ ]:
db_run_analysis.data.to_df()

In [ ]:
db_run_analysis

In [ ]:
db_run_analysis.data.to_df()